#3.2.1 Kkma

In [3]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.4 MB/s 
     |████████████████████████████████| 453 kB 57.7 MB/s 


In [4]:
from konlpy.tag import Kkma

In [5]:
kkma=Kkma()

text="커피를 줄여야 하는데"

morphs=kkma.morphs(text) # 토크나이징
print(morphs)

['커피', '를', '줄이', '어야', '하', '는데']


In [6]:
pos=kkma.pos(text) # 토크나이징 후 품사 태깅
pos

[('커피', 'NNG'),
 ('를', 'JKO'),
 ('줄이', 'VV'),
 ('어야', 'ECD'),
 ('하', 'VV'),
 ('는데', 'ECD')]

In [7]:
nouns=kkma.nouns(text) # 명사 추출
nouns

['커피']

In [11]:
sentences="오늘은 좀 피곤해. 집에 가야겠어.."
s=kkma.sentences(sentences)
s

['오늘은 좀 피곤해. 집에 가야겠어..']

---

#3.2.2 Komoran

In [12]:
|from konlpy.tag import Komoran

In [13]:
komoran=Komoran()

text="디카페인 아메리카노는 무슨 맛이지"

morphs=komoran.morphs(text)
morphs

['디', '카페인', '아메리카노', '는', '무슨', '맛', '이', '지']

In [14]:
pos=komoran.pos(text)
pos

[('디', 'NNP'),
 ('카페인', 'NNP'),
 ('아메리카노', 'NNP'),
 ('는', 'JX'),
 ('무슨', 'MM'),
 ('맛', 'NNG'),
 ('이', 'VCP'),
 ('지', 'EC')]

In [15]:
nouns=komoran.nouns(text)
nouns

['디', '카페인', '아메리카노', '맛']

---

#3.2.3 Okt

In [16]:
from konlpy.tag import Okt

In [17]:
okt=Okt()

In [18]:
text="어떻게 될까? 잘 모르겠음"
morphs=okt.morphs(text)
morphs

['어떻게', '될까', '?', '잘', '모르겠음']

In [19]:
pos=okt.pos(text)
pos

[('어떻게', 'Adjective'),
 ('될까', 'Verb'),
 ('?', 'Punctuation'),
 ('잘', 'Verb'),
 ('모르겠음', 'Verb')]

In [20]:
nouns=okt.nouns(text)
nouns

[]

In [21]:
text="개빡치넹.."
print(okt.normalize(text))
print(okt.phrases(text))

개빡치넹..
['개빡치넹']


In [22]:
text="엔엘피가 뭐죠? 반찬인가요?"
pos=komoran.pos(text)
pos

[('엔', 'NNB'),
 ('엘', 'NNP'),
 ('피', 'NNG'),
 ('가', 'JKS'),
 ('뭐', 'NP'),
 ('죠', 'EF'),
 ('?', 'SF'),
 ('반찬', 'NNG'),
 ('이', 'VCP'),
 ('ㄴ가요', 'EF'),
 ('?', 'SF')]

In [23]:
komoran=Komoran(userdic='./user_dic.tsv')
pos=komoran.pos(text)
pos

[('엔엘피', 'NNG'),
 ('가', 'JKS'),
 ('뭐', 'NP'),
 ('죠', 'EF'),
 ('?', 'SF'),
 ('반찬', 'NNG'),
 ('이', 'VCP'),
 ('ㄴ가요', 'EF'),
 ('?', 'SF')]

---
#4.2 단어 임베딩
##4.2.1 원-핫 인코딩

In [24]:
import numpy as np

komoran=Komoran()
text="오늘 학식은 쌀국수라는데?"

nouns=komoran.nouns(text)
nouns

['오늘', '학식', '쌀국수']

In [26]:
dics={}
for word in nouns:
  if word not in dics.keys():
    dics[word]=len(dics)

dics

{'오늘': 0, '학식': 1, '쌀국수': 2}

In [27]:
nb_classes=len(dics)
targets=list(dics.values())
one_hot_targets=np.eye(nb_classes)[targets]
one_hot_targets

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

---
##4.2.2 희소 표현, 분산 표현

In [28]:
from gensim.models import Word2Vec
import time

def read_review_data(filename):
  with open(filename, 'r') as f:
    data=[line.split('\t') for line in f.read().splitlines()]
    data=data[1:]

  return data

start=time.time()

print('1) 말뭉치 리드 ')
review_data=read_review_data('./ratings.txt')
print(len(review_data))
print('1) 끝',time.time()-start)

print('2) nouns 추출')
komoran=Komoran()
docs=[komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 끝',time.time()-start)

print('3) Word2Vec train')
model=Word2Vec(sentences=docs, size=200, window=4, hs=1, min_count=2, sg=1)
print('3) 끝',time.time()-start)

print('4) save')
model.save('nvmc.model')
print('4) saved',time.time()-start)

print('corpus_count:',model.corpus_count)
print('corpus_total_words:',model.corpus_total_words)

1) 말뭉치 리드 
200000
1) 끝 2.285738468170166
2) nouns 추출
2) 끝 183.31281900405884
3) Word2Vec train
3) 끝 233.74894857406616
4) save
4) saved 239.72338676452637
corpus_count: 200000
corpus_total_words: 1076896


In [29]:
print('사랑:',model.wv['사랑'])

print('일요일=월요일\t',model.wv.similarity(w1='일요일',w2='월요일'))
print('안성기=배우\t',model.wv.similarity(w1='안성기',w2='배우'))
print('대기업=삼성\t',model.wv.similarity(w1='대기업',w2='삼성'))
print('일요일!=삼성\t',model.wv.similarity(w1='일요일',w2='삼성'))
print('히어로!=삼성\t',model.wv.similarity(w1='히어로',w2='삼성'))

print(model.wv.most_similar('안성기',topn=5))
print(model.wv.most_similar('시리즈',topn=5))

사랑: [ 1.67575255e-01  4.52093557e-02 -3.99134070e-01  2.44577229e-01
 -3.13854873e-01 -9.05287489e-02 -3.10382936e-02 -1.93286017e-01
  1.89993367e-01 -7.09626153e-02  1.52033940e-01 -2.31335536e-01
 -5.70475236e-02  6.22065477e-02  2.22231150e-01  1.53397247e-01
  4.79584979e-03 -1.79485515e-01  1.92654476e-01  2.19702467e-01
 -2.43351877e-01 -1.15445256e-01  1.52711853e-01 -1.59623891e-01
  1.00487195e-01  1.58999294e-01  1.10818975e-01  7.20589533e-02
 -3.15904617e-01  1.45158902e-01  2.79480964e-01 -2.31316149e-01
 -1.07655652e-01 -1.06564842e-01  2.76942968e-01  4.35958318e-02
 -3.37973982e-01  1.30389472e-02 -3.75859708e-01 -3.97059917e-01
 -4.36505556e-01  4.51612562e-01  3.92275929e-01 -3.56177151e-01
  8.59389678e-02  2.54139751e-01 -2.77418047e-01 -1.46490887e-01
  1.24068065e-02  2.88521230e-01 -3.44899297e-02  4.57793567e-03
 -3.36047649e-01  1.17567391e-03 -9.01029855e-02 -3.41995396e-02
  9.10939574e-02 -5.06901694e-03  3.81088927e-02 -3.48933965e-01
 -8.81310701e-02 -7.7

---
#5. 텍스트 유사도
##5.1 텍스트 유사도 개요
##5.2 n-gram 유사도

In [31]:
def word_ngram(bow, num_gram):
  text=tuple(bow)
  ngrams=[text[x:x+num_gram] for x in range(0, len(text))]
  return tuple(ngrams)

def similarity(doc1, doc2):
  cnt=0

  for token in doc1:
    if token in doc2:
      cnt=cnt+1

  return cnt/len(doc1)

sentence1='6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sentence2='6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sentence3='나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

komoran=Komoran()
bow1=komoran.nouns(sentence1)
bow2=komoran.nouns(sentence2)
bow3=komoran.nouns(sentence3)

doc1=word_ngram(bow1,2)
doc2=word_ngram(bow2,2)
doc3=word_ngram(bow3,2)

print(doc1)
print(doc2)
print(doc3)

r1=similarity(doc1,doc2)
r2=similarity(doc3,doc1)

print(r1)
print(r2)

(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
(('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'), ('님과 함께',))
0.6666666666666666
0.0


---
##5.3 코사인 유사도

In [35]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(vec1, vec2):
  return dot(vec1, vec2)/(norm(vec1)*norm(vec2))

def make_term_doc_mat(sentence_bow, word_dics): # 문장별 사전 내 단어 보유 현황 리스트
  freq_mat={}

  for word in word_dics:
    freq_mat[word]=0

  for word in word_dics:
    if word in sentence_bow:
      freq_mat[word]+=1

  return freq_mat

def make_vector(tdm):
  vec=[]
  for key in tdm: # 딕셔너리 키 값이 반환됨.
    vec.append(tdm[key])

  return vec

sentence1='6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sentence2='6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sentence3='나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

bow1=komoran.nouns(sentence1)
bow2=komoran.nouns(sentence2)
bow3=komoran.nouns(sentence3)

bow=bow1+bow2+bow3

word_dics=[]

for token in bow:
  if token not in word_dics:
    word_dics.append(token)

freq_list1=make_term_doc_mat(bow1, word_dics)
freq_list2=make_term_doc_mat(bow2,word_dics)
freq_list3=make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

doc1=np.array(make_vector(freq_list1))
doc2=np.array(make_vector(freq_list2))
doc3=np.array(make_vector(freq_list3))

print('doc:',doc1)
r1=cos_sim(doc1,doc2)
r2=cos_sim(doc3,doc1)
print(r1)
print(r2)

{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1, '선생': 1, '님과 함께': 1}
doc: [1 1 1 1 1 1 0 0 0 0]
0.8333333333333335
0.20412414523193154
